In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.linalg import svd
from numpy import diag
from numpy import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error

In [2]:
#import movielens dataset
datafile='D:/cs/master/datamining/movielens/ml-100k/u.data'
data=pd.read_csv(datafile,sep="\t",header=None,names=['userID','itemID','rating','timestamp'])

In [3]:
item_cols = ['movie id','movie title']
item = pd.read_csv('D:/cs/master/datamining/movielens/ml-100k/u.item', sep='|',
names=item_cols,header=None, index_col=False,usecols=[0,1],encoding='latin-1')

In [4]:
item

,movie id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [5]:
#keep users  and item with more than 20 rate
userpermovie= data.itemID.value_counts()
data= data[data["itemID"].isin(userpermovie[userpermovie>10].index)]
movieidPeruser = data.userID.value_counts()
data= data[data["userID"].isin(movieidPeruser[movieidPeruser>20].index)]

# user clustering

In [6]:
#creat user item dataframe
userItemRatingMatrixNAN=pd.pivot_table(data, values='rating',
                                    index=['userID'], columns=['itemID'])


In [7]:
userItemRatingMatrixNAN

itemID,1,2,3,4,5,6,7,8,9,10,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
userID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# fill NAN with 0
userItemRatingMatrix=userItemRatingMatrixNAN.fillna(0)

In [9]:
#using GaussianMixture for cluster users
kmeans = KMeans(n_clusters=8, random_state=0)
userlabels=kmeans.fit(userItemRatingMatrix).predict(userItemRatingMatrix)
usercenters = kmeans.cluster_centers_

In [10]:
#creat center item data frame
usercluster=pd.DataFrame(usercenters,columns=userItemRatingMatrix.columns)


In [11]:
usercluster

itemID,1,2,3,4,5,6,7,8,9,10,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
0,2.794118,1.705882,0.470588,1.455882,0.147059,-5.551115e-17,2.044118,1.205882,0.220588,0.191176,...,4.163336e-17,7.352941e-02,8.823529e-02,1.470588e-01,-5.551115e-17,1.911765e-01,7.632783e-17,1.029412e-01,1.029412e-01,6.938894e-18
1,0.500000,0.010067,0.016779,0.036913,0.137584,3.355705e-02,0.453020,0.147651,0.553691,0.110738,...,-1.179612e-16,-3.469447e-17,-2.081668e-16,-2.081668e-16,1.342282e-02,-2.289835e-16,-2.289835e-16,1.387779e-16,4.362416e-02,2.012279e-16
2,2.928571,0.035714,0.446429,0.148810,0.113095,1.071429e-01,2.434524,0.160714,1.672619,0.398810,...,-2.775558e-17,-2.775558e-17,7.632783e-17,5.952381e-03,-1.110223e-16,7.632783e-17,-1.526557e-16,1.785714e-02,-1.734723e-16,-6.245005e-17
3,2.391304,0.304348,0.369565,2.239130,0.347826,5.869565e-01,2.673913,2.739130,2.847826,1.695652,...,1.956522e-01,1.956522e-01,8.695652e-02,6.521739e-02,7.391304e-01,-4.163336e-17,8.695652e-02,2.173913e-02,2.081668e-17,1.086957e-01
4,1.566038,0.119497,0.075472,0.729560,0.150943,1.132075e-01,1.433962,1.144654,1.075472,0.295597,...,-2.081668e-17,1.886792e-02,6.289308e-03,1.257862e-02,7.547170e-02,7.632783e-17,2.515723e-02,-3.816392e-17,3.773585e-02,-6.245005e-17
5,2.875000,2.125000,1.375000,4.062500,0.937500,3.750000e-01,3.562500,3.687500,2.937500,1.125000,...,1.000000e+00,4.375000e-01,2.500000e-01,2.500000e-01,2.500000e-01,1.875000e-01,5.625000e-01,1.875000e-01,4.375000e-01,3.750000e-01
6,3.093023,0.872093,0.383721,1.720930,0.453488,4.651163e-02,2.209302,2.430233,2.197674,0.418605,...,8.139535e-02,4.651163e-02,2.209302e-01,6.976744e-02,4.651163e-02,1.162791e-02,8.139535e-02,-5.898060e-17,3.469447e-17,3.488372e-02
7,3.655738,2.508197,1.262295,2.803279,1.967213,8.196721e-02,3.344262,2.311475,2.377049,0.655738,...,4.918033e-02,2.622951e-01,2.131148e-01,3.442623e-01,-5.551115e-17,3.442623e-01,4.098361e-01,1.803279e-01,1.311475e-01,3.606557e-01


# SVD(user cluster) 

In [12]:
#creat dataframe for each cluster
userclusters=[[],[],[],[],[],[],[],[]]
for j in range(len(usercenters)):
    for i in range(len(userlabels)):
        if userlabels[i]==j:
            userclusters[j].append(userItemRatingMatrix.iloc[i])
           

In [13]:
#SVD on each cluster and predict rating
new_userclusters=[]
for i in userclusters:
    cluster=pd.DataFrame(i)
    B=cluster.to_dense()
    B.to_numpy()
    Bmean=B.mean()
    Bstd=B.std()
    Zscore=((B-Bmean)/Bstd)
    Zscore=np.nan_to_num(Zscore)
    U,S,VT=svd(Zscore)
    s= np.diag(S[:2])
    x=np.dot(U[:,:2],s)
    Zd=np.dot(x,(VT[:,:2]).T)
    counter=0
    for j in B.columns:
        #print(j)
        counter+=1
        for k in range(len(B)):
            #print(j,k)A
            if(B[j].iloc[k]==0):
                B[j].iloc[k]=Bmean[j]+(Bstd[j]*Zd[k][counter-1])
    new_userclusters.append(B)        
    print("done",len(i))

C:\Users\sayeh\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: DataFrame/Series.to_dense is deprecated and will be removed in a future version
  """


done 68
done 298
done 168
done 46
done 159
done 16
done 86
done 61


In [14]:
new_userclusters[0]

itemID,1,2,3,4,5,6,7,8,9,10,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
5,4.000000,3.000000,0.301177,1.781723,0.290530,0.0,1.906385,1.112575,0.347049,0.084844,...,0.0,0.109830,0.086426,0.143623,0.0,0.182169,0.0,0.107488,0.062300,0.0
8,1.969651,1.320133,0.857481,2.035801,0.115880,0.0,3.000000,1.209559,0.343697,-0.014620,...,0.0,0.055490,0.013361,0.049286,0.0,0.145408,0.0,0.116750,0.029541,0.0
22,2.249509,2.000000,0.720584,5.000000,0.144403,0.0,1.572153,1.195904,0.324437,0.031731,...,0.0,0.065700,0.035193,0.077651,0.0,0.157689,0.0,0.113291,0.045707,0.0
37,1.991230,1.350063,0.864750,1.802758,0.060673,0.0,4.000000,1.248210,0.270120,0.064168,...,0.0,0.043202,0.026509,0.066908,0.0,0.156771,0.0,0.112552,0.058784,0.0
38,5.000000,2.023194,0.090830,1.289039,0.267761,0.0,2.384254,1.138632,0.224931,0.256838,...,0.0,0.110948,0.137720,0.210931,0.0,0.215940,0.0,0.096739,0.124475,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,5.000000,2.395243,-0.246855,0.550845,0.243019,0.0,4.000000,4.000000,2.000000,0.515081,...,0.0,0.115325,0.216917,0.314778,0.0,0.267512,0.0,0.080447,0.217668,0.0
901,5.000000,2.026627,0.069841,1.372362,0.295154,0.0,2.336127,3.000000,0.253961,0.229111,...,0.0,0.117549,0.134932,0.207079,0.0,0.212672,0.0,0.098087,0.114048,0.0
922,5.000000,1.734633,0.412418,1.560599,0.194717,0.0,2.002008,1.174977,0.261781,0.157094,...,0.0,0.085643,0.088042,0.146451,0.0,0.188443,0.0,0.104371,0.089886,0.0
927,5.000000,2.071711,0.096029,0.944469,0.188444,0.0,3.000000,4.000000,0.116965,0.373465,...,0.0,0.093444,0.157739,0.237725,0.0,0.232979,0.0,0.090487,0.167723,0.0


# clusters to Dataframe


In [15]:
userDataframe=[]
for i in userItemRatingMatrix.index:
    for j in range(len(usercenters)):
        for k in new_userclusters[j].index:
            if(i==k):
                userDataframe.append(new_userclusters[j].loc[k])   
userDataframe=pd.DataFrame(userDataframe)                

In [16]:
userDataframe

itemID,1,2,3,4,5,6,7,8,9,10,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
1,5.000000,3.000000,4.000000,3.000000,3.000000,5.000000,4.000000,1.000000,5.000000,3.000000,...,0.08531,0.184628,0.129506,0.208368,0.000000,0.633619,0.526915,0.320388,0.238212,0.370763
2,4.000000,0.010015,0.515029,0.114028,0.117942,0.185122,2.162334,0.055960,1.346953,2.000000,...,0.00000,0.000000,0.000000,0.006259,0.000000,0.000000,0.000000,0.010493,0.000000,0.000000
3,0.505237,0.013503,0.004473,0.054351,0.103335,0.038969,0.450777,0.184890,0.545139,0.160143,...,0.00000,0.000000,0.000000,0.000000,0.027676,0.000000,0.000000,0.000000,0.095281,0.000000
4,0.502658,0.010788,0.014696,0.041172,0.134599,0.035477,0.453582,0.158900,0.569442,0.116760,...,0.00000,0.000000,0.000000,0.000000,0.017567,0.000000,0.000000,0.000000,0.062931,0.000000
5,4.000000,3.000000,0.301177,1.781723,0.290530,0.000000,1.906385,1.112575,0.347049,0.084844,...,0.00000,0.109830,0.086426,0.143623,0.000000,0.182169,0.000000,0.107488,0.062300,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,2.921126,0.023278,0.451334,0.137820,0.169402,0.123272,2.291070,0.200985,5.000000,0.272174,...,0.00000,0.000000,0.000000,0.005412,0.000000,0.000000,0.000000,0.018414,0.000000,0.000000
940,1.565797,0.123576,0.066490,2.000000,0.165713,0.079863,4.000000,5.000000,3.000000,0.343519,...,0.00000,0.019573,0.005197,0.011953,0.080092,0.000000,0.036654,0.000000,0.037072,0.000000
941,5.000000,0.010878,0.012553,0.039443,0.118395,0.032762,4.000000,0.147365,0.505307,0.133883,...,0.00000,0.000000,0.000000,0.000000,0.013740,0.000000,0.000000,0.000000,0.033442,0.000000
942,1.677591,0.107374,0.044740,0.707675,0.288320,-0.005377,1.083968,1.012040,1.106402,0.514309,...,0.00000,0.014304,0.000790,0.008426,0.109150,0.000000,0.047921,0.000000,0.031275,0.000000


# similarity (usercluster)

In [17]:
usersim=[]
for cluster in new_userclusters:
    usersim.append(cosine_similarity(cluster))
for i in range(len(usersim)): 
    usersim[i]=pd.DataFrame(usersim[i])

In [18]:
usersim[0]

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,1.000000,0.752699,0.745580,0.765287,0.573083,0.682178,0.684087,0.526113,0.712938,0.655760,...,0.652913,0.703118,0.649094,0.657019,0.679786,0.514704,0.600629,0.701533,0.604179,0.647866
1,0.752699,1.000000,0.763193,0.802894,0.571853,0.661552,0.730405,0.558579,0.730953,0.633714,...,0.693195,0.741063,0.661633,0.642710,0.680566,0.515474,0.658472,0.703735,0.595911,0.605137
2,0.745580,0.763193,1.000000,0.757340,0.562234,0.693148,0.684336,0.582540,0.705176,0.630832,...,0.638092,0.709076,0.643322,0.619026,0.667094,0.553710,0.653270,0.700794,0.593548,0.634914
3,0.765287,0.802894,0.757340,1.000000,0.611793,0.733053,0.721917,0.648664,0.720379,0.707575,...,0.670417,0.763915,0.673720,0.641128,0.725215,0.609020,0.678982,0.731260,0.680270,0.718431
4,0.573083,0.571853,0.562234,0.611793,1.000000,0.653283,0.621505,0.648562,0.600520,0.612030,...,0.562560,0.640920,0.581059,0.517881,0.660559,0.626332,0.651975,0.632969,0.658911,0.576459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.514704,0.515474,0.553710,0.609020,0.626332,0.643796,0.627288,0.656777,0.570426,0.563364,...,0.520105,0.640034,0.597499,0.558670,0.645227,1.000000,0.611295,0.619592,0.676494,0.603041
64,0.600629,0.658472,0.653270,0.678982,0.651975,0.691087,0.671599,0.721345,0.653627,0.634476,...,0.606191,0.675809,0.632892,0.569155,0.691356,0.611295,1.000000,0.681358,0.691545,0.620081
65,0.701533,0.703735,0.700794,0.731260,0.632969,0.732739,0.715627,0.650599,0.684204,0.667524,...,0.654479,0.747567,0.668821,0.644296,0.726726,0.619592,0.681358,1.000000,0.700879,0.658762
66,0.604179,0.595911,0.593548,0.680270,0.658911,0.706674,0.676829,0.695919,0.628341,0.658665,...,0.569925,0.690154,0.637882,0.557430,0.704551,0.676494,0.691545,0.700879,1.000000,0.661550


# item clustering

In [19]:
#creat item user dataframe
itemUserRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['itemID'], columns=['userID'])

In [20]:
itemUserRatingMatrix

userID,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
itemID,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# fill NAN with 0
itemUserRatingMatrix=itemUserRatingMatrix.fillna(0)

In [22]:
#using GaussianMixture for cluster items
kmeans = KMeans(n_clusters=8, random_state=0)
itemlabels=kmeans.fit(itemUserRatingMatrix).predict(itemUserRatingMatrix)
itemcenters =kmeans.cluster_centers_

In [23]:
#creat center item data frame
itemcluster=pd.DataFrame(itemcenters,columns=itemUserRatingMatrix.columns).T

In [24]:
itemcluster


,0,1,2,3,4,5,6,7
userID,,,,,,,,
1,0.235202,1.175000e+00,3.326087e+00,2.000000,1.690141,5.853659e-01,1.900000e+00,4.750
2,0.099688,1.250000e-01,-1.110223e-16,1.566667,0.091549,1.634146e+00,-1.387779e-16,1.875
3,0.105919,1.250000e-02,8.695652e-02,0.200000,0.000000,1.634146e+00,0.000000e+00,0.500
4,0.052960,1.804112e-16,1.521739e-01,0.466667,0.021127,6.585366e-01,8.000000e-02,0.625
5,0.182243,3.500000e-01,1.586957e+00,0.866667,1.056338,-2.775558e-16,7.400000e-01,3.375
...,...,...,...,...,...,...,...,...
939,0.119938,6.250000e-02,-1.110223e-16,2.666667,0.190141,2.439024e-01,-1.110223e-16,0.625
940,0.040498,3.875000e-01,1.695652e+00,0.933333,0.239437,1.585366e+00,1.000000e+00,3.375
941,0.015576,9.375000e-02,-2.775558e-17,1.466667,0.077465,9.756098e-02,-4.163336e-17,0.625


In [25]:
len(itemlabels)

1119

# SVD(item cluster)

In [26]:
#creat dataframe for each cluster
itemclusters=[[],[],[],[],[],[],[],[]]
for j in range(len(itemcenters)):
    for i in range(len(itemlabels)):
        if itemlabels[i]==j:
            itemclusters[j].append(itemUserRatingMatrix.iloc[i])
           

In [27]:
pd.DataFrame(itemclusters[0])

userID,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
6,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,5.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
21,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#SVD on each cluster and predict rating
new_itemclusters=[]
for i in itemclusters:
    cluster=pd.DataFrame(i)
    B=cluster.to_dense()
    B.to_numpy()
    Bmean=B.mean()
    Bstd=B.std()
    Zscore=((B-Bmean)/Bstd)
    Zscore=np.nan_to_num(Zscore)
    U,S,VT=svd(Zscore)
    s= np.diag(S[:2])
    x=np.dot(U[:,:2],s)
    Zd=np.dot(x,(VT[:,:2]).T)
    counter=0
    for j in B.columns:
        #print(j)
        counter+=1
        for k in range(len(B)):
            #print(j,k)A
            if(B[j].iloc[k]==0):
                B[j].iloc[k]=Bmean[j]+(Bstd[j]*Zd[k][counter-1])
    new_itemclusters.append(B)        
    print("done",len(i))

C:\Users\sayeh\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: DataFrame/Series.to_dense is deprecated and will be removed in a future version
  """


done 642
done 160
done 46
done 30
done 142
done 41
done 50
done 8


In [29]:
new_itemclusters[0]

userID,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
6,5.000000,0.106989,0.100376,0.044156,0.178750,0.086000,0.521556,0.037497,5.000000,0.088372,...,0.156262,0.041209,5.000000,0.043515,0.196286,0.077335,0.037589,0.004366,0.063967,0.194588
16,5.000000,0.154522,0.235183,0.109244,0.083703,0.093626,0.522865,0.062513,0.012168,4.000000,...,0.259433,0.095870,4.000000,0.180030,0.145380,0.304983,0.052287,0.030267,0.045335,0.105126
19,5.000000,3.000000,0.918795,0.363423,-0.489947,4.000000,0.449898,0.145647,0.010716,0.282956,...,0.455503,0.274654,5.000000,1.000000,0.054530,1.047685,0.098269,0.035965,0.019819,-0.260489
21,1.000000,0.152200,0.363725,0.202850,3.000000,3.000000,0.557202,0.104542,-0.040281,0.182738,...,0.493781,0.188893,0.881486,0.394077,0.027300,0.693190,0.078169,0.107194,-0.001099,-0.016777
27,2.000000,0.063796,0.106001,0.076780,0.210913,0.212436,4.000000,0.055774,-0.005289,0.106976,...,0.289492,0.082273,0.394830,0.128741,0.128226,0.249102,0.049455,0.054969,0.036007,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,0.027130,0.009721,-0.065079,-0.009957,0.326537,0.185315,0.546985,0.021643,0.021415,0.054953,...,0.140731,0.007541,0.062513,-0.055505,0.201948,-0.062242,0.029246,0.015250,0.063412,0.274486
1478,0.038018,0.011784,-0.058224,-0.006413,0.321986,0.186690,0.547297,0.023050,0.020267,0.057076,...,0.146987,0.010625,0.076037,-0.047941,0.198843,-0.049395,0.030082,0.016954,0.062252,0.269665
1480,-0.004421,0.004412,-0.084413,-0.020291,0.338961,0.180009,0.545750,0.017480,0.025038,0.048772,...,0.121628,-0.001593,0.023344,-0.077740,0.211451,-0.100310,0.026762,0.009881,0.066992,0.288475
1483,-0.011209,0.040390,-0.059194,-0.026047,0.299520,0.105821,0.527184,0.011827,0.042219,0.045920,...,0.063631,-0.014621,0.016001,-0.099885,0.241461,-0.155013,0.022765,-0.014965,0.079804,0.292546


In [30]:
itemDataframe=[]
for i in itemUserRatingMatrix.index:
    for j in range(len(itemcenters)):
        for k in new_itemclusters[j].index:
            if(i==k):
                itemDataframe.append(new_itemclusters[j].loc[k])   
itemDataframe=pd.DataFrame(itemDataframe).T  

In [31]:
itemDataframe

,1,2,3,4,5,6,7,8,9,10,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
userID,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.000000,4.000000,3.000000,3.000000,5.000000,4.000000,1.000000,5.000000,3.000000,...,0.022725,0.044437,0.021619,0.062732,-0.060684,0.027130,0.038018,-0.004421,-0.011209,0.029024
2,4.000000,-0.153547,0.316114,0.000000,0.026438,0.106989,1.694166,0.000000,1.661884,2.000000,...,0.016062,0.022871,0.005329,0.018304,0.002433,0.009721,0.011784,0.004412,0.040390,0.013664
3,-0.045141,0.000000,0.000000,0.108580,0.000000,0.100376,0.088669,0.363034,0.602010,0.036913,...,-0.062173,-0.046371,-0.071198,-0.041210,-0.112963,-0.065079,-0.058224,-0.084413,-0.059194,-0.061050
4,0.095015,0.021461,0.056220,0.144647,0.014545,0.044156,0.468663,0.056077,0.652142,0.000000,...,-0.012074,-0.005263,-0.011433,0.001456,-0.039430,-0.009957,-0.006413,-0.020291,-0.026047,-0.009682
5,4.000000,3.000000,1.107813,1.572744,1.052805,0.178750,0.669966,1.405500,1.783345,0.173401,...,0.320236,0.308105,0.332637,0.309574,0.352623,0.326537,0.321986,0.338961,0.299520,0.321679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,2.886335,0.145479,0.250846,0.000000,0.174548,0.077335,2.641470,0.000000,5.000000,0.079458,...,-0.076432,-0.054190,-0.064551,-0.022537,-0.177574,-0.062242,-0.049395,-0.100310,-0.155013,-0.064499
940,-0.799908,0.214450,0.172049,2.000000,0.249811,0.037589,4.000000,5.000000,3.000000,-0.011337,...,0.028239,0.029653,0.029135,0.031807,0.021635,0.029246,0.030082,0.026762,0.022765,0.029057
941,5.000000,0.031850,0.141913,0.000000,0.061078,0.004366,4.000000,0.000000,1.797002,-0.013989,...,0.009980,0.011660,0.016523,0.019652,-0.004471,0.015250,0.016954,0.009881,-0.014965,0.013259


# similarity(itemcluster)

In [32]:
itemsim=[]
for cluster in new_itemclusters:
    itemsim.append(cosine_similarity(cluster))
for i in range(len(itemsim)): 
    itemsim[i]=pd.DataFrame(itemsim[i])

In [33]:
itemsim[0]

,0,1,2,3,4,5,6,7,8,9,...,632,633,634,635,636,637,638,639,640,641
0,1.000000,0.301882,0.304847,0.246895,0.247313,0.300551,0.365825,0.235643,0.242044,0.277321,...,0.370858,0.243024,0.242887,0.248410,0.230151,0.257187,0.308851,0.293758,0.230559,0.262712
1,0.301882,1.000000,0.343528,0.288941,0.275606,0.278856,0.331449,0.355630,0.245468,0.263667,...,0.253372,0.246349,0.250059,0.289397,0.179991,0.237990,0.230834,0.305572,0.211365,0.295477
2,0.304847,0.343528,1.000000,0.207906,0.143083,0.143217,0.188956,0.150773,0.132437,0.142585,...,0.170583,0.118086,0.110618,0.114733,0.097196,0.103829,0.136950,0.131936,0.116076,0.133021
3,0.246895,0.288941,0.207906,1.000000,0.326544,0.356972,0.398355,0.390855,0.341316,0.327162,...,0.322009,0.349699,0.345362,0.375029,0.229847,0.439573,0.339148,0.381215,0.366745,0.296240
4,0.247313,0.275606,0.143083,0.326544,1.000000,0.354540,0.436025,0.315178,0.404092,0.343119,...,0.363694,0.364267,0.355179,0.418953,0.221999,0.369196,0.388745,0.423131,0.312852,0.326734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,0.257187,0.237990,0.103829,0.439573,0.369196,0.530122,0.562483,0.426536,0.500069,0.376074,...,0.403965,0.475991,0.491501,0.491332,0.309277,1.000000,0.415980,0.598950,0.406081,0.498158
638,0.308851,0.230834,0.136950,0.339148,0.388745,0.428007,0.496967,0.417180,0.429741,0.393794,...,0.437503,0.423116,0.403133,0.416617,0.339812,0.415980,1.000000,0.463665,0.444023,0.535187
639,0.293758,0.305572,0.131936,0.381215,0.423131,0.629714,0.551591,0.480809,0.439088,0.479480,...,0.511076,0.506383,0.567077,0.509326,0.344586,0.598950,0.463665,1.000000,0.409162,0.550495
640,0.230559,0.211365,0.116076,0.366745,0.312852,0.380620,0.437710,0.298723,0.345996,0.317993,...,0.339488,0.432383,0.394881,0.346430,0.269065,0.406081,0.444023,0.409162,1.000000,0.414498


# combine user and item cluster

In [67]:
a=0.3
a *(userDataframe[3].loc[2]) + (1-a)*(itemDataframe[3].loc[2])

0.3757883114615431

In [68]:
df1=(((userDataframe-userDataframe.min())/(userDataframe.max()-userDataframe.min()))*4)+1
df2=(((itemDataframe-itemDataframe.min())/(itemDataframe.max()-itemDataframe.min()))*4)+1

In [69]:
for i in userItemRatingMatrix.index:
    for j in userItemRatingMatrix.columns:
        if(userItemRatingMatrix[j].loc[i]==0):
            userItemRatingMatrix[j].loc[i]= a *(df1[j].loc[i]) + (1-a)*(df2[j].loc[i])
        else:    
            userItemRatingMatrix[j].loc[i]=0
            

In [41]:
predicte_userItemRatingMatrix=userItemRatingMatrix

In [42]:
predicte_userItemRatingMatrix

itemID,1,2,3,4,5,6,7,8,9,10,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
userID,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.099919,1.111636,1.127070,1.129376,1.112217,1.146849,1.252579,1.134289,1.146949,1.150084
2,0.000000,1.060834,1.682333,1.058905,1.104349,1.148229,2.356176,1.519970,2.544715,0.000000,...,1.088622,1.082031,1.100022,1.082099,1.156103,1.085455,1.189779,1.112118,1.165300,1.100758
3,1.911945,1.167991,1.445688,1.132916,1.084408,1.133435,1.064714,1.754048,1.820208,1.375323,...,1.033281,1.032915,1.032854,1.039474,1.077910,1.032586,1.128189,1.033236,1.102828,1.034913
4,1.987319,1.182742,1.482838,1.157964,1.096949,1.092738,1.338532,1.560358,1.853797,1.348076,...,1.068719,1.062074,1.085310,1.069681,1.128291,1.071546,1.173770,1.089449,1.123576,1.080183
5,0.000000,0.000000,2.183551,2.307704,1.851830,1.187160,1.598489,2.454618,2.573243,1.465544,...,1.303784,1.292872,1.395812,1.299067,1.399592,1.323497,1.462684,1.414766,1.350888,1.372208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,3.677703,1.269972,1.636030,1.060566,1.214010,1.122643,3.048427,1.527359,0.000000,1.409498,...,1.023195,1.027368,1.038688,1.053118,1.030943,1.034591,1.135956,1.021077,1.028841,1.031874
940,1.586244,1.325047,1.561259,0.000000,1.267717,1.091059,0.000000,0.000000,0.000000,1.350430,...,1.097235,1.088359,1.121428,1.092105,1.175366,1.099255,1.208640,1.130699,1.155744,1.114323
941,0.000000,1.189983,1.538376,1.053698,1.129214,1.063917,0.000000,1.524627,2.589963,1.339249,...,1.084319,1.074078,1.109846,1.082563,1.152317,1.089363,1.194328,1.115899,1.129126,1.100401


In [43]:
userItemRatingMatrix=userItemRatingMatrixNAN.fillna(0)

# Recommend movie

In [44]:
def recommend(targetUser):
    RecommeMovieList=[]
    predicte_userItemRatingMatrix.iloc[targetUser]
    TopRating=predicte_userItemRatingMatrix.loc[targetUser]
    TopRating.sort_values(ascending=False)[:5].index
    movieID=(TopRating.sort_values(ascending=False)[:5].index)
    for i in movieID:
        RecommeMovieList.append(item[item["movie id"]==i]["movie title"])
    return RecommeMovieList    

In [45]:
recommend(200)

[96    Dances with Wolves (1990)
 Name: movie title, dtype: object, 69    Four Weddings and a Funeral (1994)
 Name: movie title, dtype: object, 180    Return of the Jedi (1983)
 Name: movie title, dtype: object, 215    When Harry Met Sally... (1989)
 Name: movie title, dtype: object, 650    Glory (1989)
 Name: movie title, dtype: object]

# evaluate

In [46]:
#create test data set
test=[]
randlist=[]
count=0
while(count!=100):
    rand=np.random.randint(1,902)
    if(rand not in randlist):
        for user in userItemRatingMatrix.index:
            if(user==rand):
            #print(rand)
                test.append(userItemRatingMatrix.loc[user]) 
                count+=1
                randlist.append(rand)
test=pd.DataFrame(test)  
    

In [47]:
test

itemID,1,2,3,4,5,6,7,8,9,10,...,1421,1428,1444,1446,1451,1469,1478,1480,1483,1518
453,0.0,0.0,4.0,4.0,0.0,0.0,5.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
716,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
728,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,4.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
792,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## method A(user and item clustering+ SVD)

In [48]:
pred=[]
real=[]
for i in test.index:
    for j in test.columns:
        if(test[j].loc[i]!=0):
            real.append(test[j].loc[i])
            pred.append(a *(df1[j].loc[i]) + (1-a)*(df2[j].loc[i]))
        else:    
            test[j].loc[i]=0

In [49]:
len(real)

9270

In [237]:
'''' Relevant items are already known in the data set
Relevant item: Has a True/Actual rating >= 3.5
Irrelevant item: Has a True/Actual rating < 3.5
# Recommended items are generated by recommendation algorithm
Recommended item: has a predicted rating >= 3.5
Not recommended item: Has a predicted rating < 3.5''''


SyntaxError: EOL while scanning string literal (<ipython-input-237-268758bc8ec6>, line 6)

In [50]:
TR=0
TN=0
FR=0
FN=0
for i in range(len(pred)):

    if(real[i]>=3.5 and pred[i]>=3.5):
        TR+=1
    elif(real[i]<3.5 and pred[i]<3.5):
        TN+=1
    elif(real[i]<3.5 and pred[i]>=3.5):
        FR+=1
    else:
        FN+=1
           

In [51]:
# MAE
mean_absolute_error(real,pred)

0.4221903937542532

In [52]:
#Precision
precision=TR/(TR+FR)
precision

0.7934704444124838

In [53]:
#recall
recall=TR/(TR+FN)
recall

0.9806256665481692

In [54]:
#F1
F1=(2*(precision*recall))/(precision+recall)
F1

0.8771762461244932